In [72]:
import os


In [73]:
os.getcwd()

'c:\\Users\\prana\\machine_learning_project_1\\ml_projects'

In [74]:
os.chdir("c:\\Users\\prana\\machine_learning_project_1\\ml_projects")

In [75]:
from housing.config.configuration import configuration

In [76]:
config=configuration()

In [77]:
config.get_data_transformation_config()

DataTransformationconfig(add_bedroom_per_room=True, tranformed_train_dir='c:\\Users\\prana\\machine_learning_project_1\\ml_projects\\housing\\artifact\\data_transformation\\2022-07-08_14-22-10\\transformed_data\\train', transformed_test_dir='c:\\Users\\prana\\machine_learning_project_1\\ml_projects\\housing\\artifact\\data_transformation\\2022-07-08_14-22-10\\transformed_data\\test', preprocessed_object_file_path='c:\\Users\\prana\\machine_learning_project_1\\ml_projects\\housing\\artifact\\data_transformation\\2022-07-08_14-22-10\\preprocessed\\preprocessed.pkl')

In [78]:
from housing.entity.artifact_entity import DataValidationArtifact

In [79]:
train_file_path=r"C:\Users\prana\machine_learning_project_1\ml_projects\housing\artifact\data_ingestion\2022-07-01_00-14-17\ingested_data\train\housing.csv"

In [80]:
import pandas as pd
import  numpy as np

In [81]:
df=pd.read_csv(train_file_path)

In [82]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,72100.0,INLAND
1,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,279600.0,NEAR OCEAN
2,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,82700.0,INLAND
3,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,112500.0,NEAR OCEAN
4,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,238300.0,<1H OCEAN


In [83]:
df.shape

(16512, 10)

In [84]:
x,y=df.drop("median_house_value",axis=1),df[["median_house_value"]]
# this code will split the data into independent and dependent feature

In [85]:
y.shape

(16512, 1)

In [86]:
x.shape

(16512, 9)

In [87]:
x.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        158
population              0
households              0
median_income           0
ocean_proximity         0
dtype: int64

In [88]:
from sklearn.impute import SimpleImputer

In [89]:
numerical_column=x.columns.drop("ocean_proximity")

In [90]:
numerical_data=df[numerical_column]

In [91]:
simple_imputer=SimpleImputer(strategy="median")

In [92]:
df=simple_imputer.fit_transform(numerical_data)

In [93]:
simple_imputer.feature_names_in_
# This will give the feature used for the transformation

array(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype=object)

In [94]:
simple_imputer.statistics_
# This will give the medisn of each and every column

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

In [95]:
x.ocean_proximity.value_counts()

<1H OCEAN     7277
INLAND        5262
NEAR OCEAN    2124
NEAR BAY      1847
ISLAND           2
Name: ocean_proximity, dtype: int64

In [96]:
simple_imputer=SimpleImputer(strategy="most_frequent")

In [97]:
simple_imputer.fit_transform(y)

array([[ 72100.],
       [279600.],
       [ 82700.],
       ...,
       [140400.],
       [258100.],
       [ 62700.]])

In [98]:
# custome Transformer --> This is just to show that we can als0 crate our own transformer whenever we need
class FeatureTransformer:
    def __init__(self,stratgy="median"):
        self.stratgy=stratgy

    def fit(self,x):
        self.feature=x.columns
        self.ststistics_=[]
        for column in x.columns:
            self.ststistics_.append(x[column].median())
            
    def transform(self,x:pd.DataFrame):
        for idx,column in enumerate( x.columns):
            x[column].fillna(self.ststistics_[idx])
        return x
        
    def fit_transform(self,x):
        self.fit(x)
        return self.transform(x)




In [99]:
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


In [100]:

# This class is created to generate new feature 
class FeatureGenerator(BaseEstimator, TransformerMixin):
    
    def __init__(self, add_bedrooms_per_room=True,
                 total_rooms_ix=3,
                 population_ix=5,
                 households_ix=6,
                 total_bedrooms_ix=4, 
                 columns=None):
        """
        FeatureGenerator Initialization
        add_bedrooms_per_room: bool
        total_rooms_ix: int index number of total rooms columns
        population_ix: int index number of total population columns
        households_ix: int index number of  households columns
        total_bedrooms_ix: int index number of bedrooms columns
        """
        try:
            # This will be stored inside the constannt folder ,just to visualize i have shown here
            COLUMN_TOTAL_ROOMS = "total_rooms"
            COLUMN_POPULATION = "population"
            COLUMN_HOUSEHOLDS = "households"
            COLUMN_TOTAL_BEDROOM = "total_bedrooms"
            DATASET_SCHEMA_COLUMNS_KEY=  "columns"
            
            self.columns = columns
            if self.columns is not None:
                total_rooms_ix = self.columns.index(COLUMN_TOTAL_ROOMS) 
                population_ix = self.columns.index(COLUMN_POPULATION)
                households_ix = self.columns.index(COLUMN_HOUSEHOLDS)
                total_bedrooms_ix = self.columns.index(COLUMN_TOTAL_BEDROOM)

            self.add_bedrooms_per_room = add_bedrooms_per_room
            self.total_rooms_ix = total_rooms_ix
            self.population_ix = population_ix
            self.households_ix = households_ix
            self.total_bedrooms_ix = total_bedrooms_ix
        except Exception as e:
            print(e)
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        """ This will just create extra column NOTHING ELSE"""
        try:
            room_per_household = X[:, self.total_rooms_ix] / X[:, self.households_ix]
            population_per_household = X[:, self.population_ix] /  X[:, self.households_ix]
            if self.add_bedrooms_per_room:
                bedrooms_per_room = X[:, self.total_bedrooms_ix] / X[:, self.total_rooms_ix]
                generated_feature = np.c_[X, room_per_household, population_per_household, bedrooms_per_room]
            else:
                generated_feature = np.c_[X, room_per_household, population_per_household]

            return generated_feature
        except Exception as e:
            print(e)





In [101]:
numerical_column

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype='object')

In [102]:
# see this is the technique where we will create a automatic transformation pipeline for the numerical as well as categorical data
# for this we have a module called Pileline and inside that we have steps , which will direct what to do first and what not to do

#steps will be like:
    #1) we will apply simple imputer so that we can replace null values with whatever stsrtegy we can
    #2) we will generate the new feature based on our custome feature generator
    #3)and at last we will use standard scaler

num_pipeline = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy="median")),
                ('feature_generator', FeatureGenerator()),
                ('scaler', StandardScaler())]
            )


# same this is for categorical feature , the steps of transformation will not be same fo both the numerical and categorical feature
#steps will be like:
    #1) in this case we will use imputer stsrtegy as most_frequent because it is a categorical column
    #2) encoder wise we will use one hot encoding, we will not use our featuregenerator here because this is a categorical column
    #3) in this we will use standard scaler at the last
    
cat_pipeline = Pipeline(steps=[
                 ('impute', SimpleImputer(strategy="most_frequent")),
                 ('one_hot_encoder', OneHotEncoder()),
                 ('scaler', StandardScaler(with_mean=False))
            ]
            )

In [103]:
cat_pipeline

Pipeline(steps=[('impute', SimpleImputer(strategy='most_frequent')),
                ('one_hot_encoder', OneHotEncoder()),
                ('scaler', StandardScaler(with_mean=False))])

In [104]:
num_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('feature_generator', FeatureGenerator()),
                ('scaler', StandardScaler())])

In [105]:
cat_column=["ocean_proximity"]

In [106]:
from sklearn.compose import ColumnTransformer


In [107]:
# see this beautiful thing , basically we have created the pipeline for transformation of our various column
# now we will have to use  the ColumnTransformer to integrate all these pipeline
# now this preprocessing object is ready , so for any transformation we will directly use it we will see it later in the code
preprocessing=ColumnTransformer([
    ("num_pipeline",num_pipeline,numerical_column),
    ("cat_pipeline",cat_pipeline,cat_column)]
)

In [108]:
x

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,INLAND
1,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,NEAR OCEAN
2,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,INLAND
3,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,NEAR OCEAN
4,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,<1H OCEAN
...,...,...,...,...,...,...,...,...,...
16507,-117.07,33.03,14.0,6665.0,1231.0,2026.0,1001.0,5.0900,<1H OCEAN
16508,-121.42,38.51,15.0,7901.0,1422.0,4769.0,1418.0,2.8139,INLAND
16509,-122.72,38.44,48.0,707.0,166.0,458.0,172.0,3.1797,<1H OCEAN
16510,-122.70,38.31,14.0,3155.0,580.0,1208.0,501.0,4.1964,<1H OCEAN


In [109]:
# now this preprocessing object have fit_transform--> which will directly take the data and returnd the transformed data
# see how beautiful it is , like we just have to pass the data and every thing will be generate according the pipeline we created
# this x containing the training dataset 
train_df=preprocessing.fit_transform(x)

In [110]:
train_df

array([[-0.94135046,  1.34743822,  0.02756357, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.17178212, -1.19243966, -1.72201763, ...,  0.        ,
         0.        ,  2.9869105 ],
       [ 0.26758118, -0.1259716 ,  1.22045984, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.5707942 ,  1.31001828,  1.53856552, ...,  0.        ,
         0.        ,  0.        ],
       [-1.56080303,  1.2492109 , -1.1653327 , ...,  0.        ,
         0.        ,  0.        ],
       [-1.28105026,  2.02567448, -0.13148926, ...,  0.        ,
         0.        ,  0.        ]])

In [111]:
# lets tranform our testing dataset

test_file_path=r"C:\Users\prana\machine_learning_project_1\ml_projects\housing\artifact\data_ingestion\2022-07-01_00-14-17\ingested_data\test\housing.csv"

In [112]:
test_df=pd.read_csv(test_file_path)

In [113]:
test_df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-118.39,34.12,29.0,6447.0,1012.0,2184.0,960.0,8.2816,500001.0,<1H OCEAN
1,-120.42,34.89,24.0,2020.0,307.0,855.0,283.0,5.0099,162500.0,<1H OCEAN
2,-118.45,34.25,36.0,1453.0,270.0,808.0,275.0,4.3839,204600.0,<1H OCEAN
3,-118.10,33.91,35.0,1653.0,325.0,1072.0,301.0,3.2708,159700.0,<1H OCEAN
4,-117.07,32.77,38.0,3779.0,614.0,1495.0,614.0,4.3529,184000.0,NEAR OCEAN
...,...,...,...,...,...,...,...,...,...,...
4123,-117.29,33.72,19.0,2248.0,427.0,1207.0,368.0,2.8170,110000.0,<1H OCEAN
4124,-118.24,33.99,33.0,885.0,294.0,1270.0,282.0,2.1615,118800.0,<1H OCEAN
4125,-119.72,34.44,43.0,1781.0,342.0,663.0,358.0,4.7000,293800.0,<1H OCEAN
4126,-117.91,33.63,30.0,2071.0,412.0,1081.0,412.0,4.9125,335700.0,<1H OCEAN


In [114]:
# in this way we will gwt our transformed test dataset
preprocessing.transform(test_df)

array([[ 0.59229422, -0.71065803,  0.02756357, ...,  0.        ,
         0.        ,  0.        ],
       [-0.42180959, -0.35049119, -0.37006852, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.56232071, -0.64985064,  0.5842485 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.07211862, -0.56097831,  1.14093342, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.83208232, -0.93985512,  0.10708999, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.50736927, -0.67791559,  0.5842485 , ...,  0.        ,
         0.        ,  0.        ]])

In [115]:
# now this is what we called preprocessing pickle file , we will store our preprocessing object inside the file
# to store thet we will either use pickle library or dill library in this case i will use dill library
import dill



In [116]:
# we are saving it in preprocessing.pkl file and whwnever we requires it we will just read this
with open("Preprocessing.pkl","wb") as prep_file:
    dill.dump(preprocessing,prep_file)

In [117]:
# now we can read it  whenever it will be required
with open("Preprocessing.pkl","rb") as read_file:
    preprocessing_obj_load=dill.load(read_file)

In [118]:
# by reading the pickle object we can transform our data
preprocessing_obj_load.transform(test_df)

array([[ 0.59229422, -0.71065803,  0.02756357, ...,  0.        ,
         0.        ,  0.        ],
       [-0.42180959, -0.35049119, -0.37006852, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.56232071, -0.64985064,  0.5842485 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.07211862, -0.56097831,  1.14093342, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.83208232, -0.93985512,  0.10708999, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.50736927, -0.67791559,  0.5842485 , ...,  0.        ,
         0.        ,  0.        ]])

In [119]:
df=pd.read_csv(train_file_path)
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,72100.0,INLAND
1,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,279600.0,NEAR OCEAN
2,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,82700.0,INLAND
3,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,112500.0,NEAR OCEAN
4,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,238300.0,<1H OCEAN
...,...,...,...,...,...,...,...,...,...,...
16507,-117.07,33.03,14.0,6665.0,1231.0,2026.0,1001.0,5.0900,268500.0,<1H OCEAN
16508,-121.42,38.51,15.0,7901.0,1422.0,4769.0,1418.0,2.8139,90400.0,INLAND
16509,-122.72,38.44,48.0,707.0,166.0,458.0,172.0,3.1797,140400.0,<1H OCEAN
16510,-122.70,38.31,14.0,3155.0,580.0,1208.0,501.0,4.1964,258100.0,<1H OCEAN


In [120]:
from sklearn.linear_model import LinearRegression

In [121]:
X=train_df

In [122]:
# but the training datasetis in the dorm of array so we need to convart this into array
y=np.array(df.median_house_value)

In [123]:

linear_regression=LinearRegression()

In [124]:
linear_regression.fit(X,y)

LinearRegression()

In [125]:
y_pred=linear_regression.predict(X)

In [127]:
from sklearn.metrics import r2_score

In [128]:
r2_score(y,y_pred)

0.648155362528312